In [1]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json

import math
import datetime

from sklearn import preprocessing, cross_validation, svm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVC

from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt

import xgboost

/Users/yoo/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# read train data
with open('./data/train_data.json') as fp:
    json_str = fp.read()
    json_data = json.loads(json_str)

In [3]:
train_df = pd.DataFrame(json_data, columns=['date','name','open','close','low','high','vol'])

In [4]:
with open('./data/grading.input.txt') as fp:
    sbm_in = fp.read()
    sbm_in = sbm_in.splitlines()

In [112]:
# teller_df = train_df[train_df['name']=='teller']
pred_lst = []
total = 0
true_t = 0
for idx, name in enumerate(sbm_in[:]):

    teller_df = train_df[train_df['name']==name]
    teller_df = teller_df.set_index('date')

    teller_df['hl_pct'] = (teller_df['high'] - teller_df['low'])/(teller_df['low']*100)
    teller_df['pct_chng'] = (teller_df['close'] - teller_df['open'])/(teller_df['open']*100)
    


    # 예측 컬럼
    forecast_col = 'close'
    # 예측 기간 하루
    forecast_out = int(1)
    # forecast_out에 맞춰 label을 변경
    teller_df['label'] = teller_df[forecast_col].shift(-forecast_out)
    # 방향 예측을 위한 전처리
    teller_df['direction'] = teller_df['label'] >= teller_df['close']

    # 과거 close
    k = 21
    for i in range(k):
        teller_df['before_'+str(i)] = teller_df[forecast_col].shift(forecast_out+i)

    
    teller_df = teller_df[k:]
    
    # X, y 'hl_pct','pct_chng',
    X_before = teller_df.drop(['name', 'open','low','high',
                               'vol','label','direction'],
                              axis=1)
    X = preprocessing.scale(X_before)
    X_forecast_out = X[-forecast_out:]
    X = X[:-forecast_out]

    #X_d = X_before.drop(['hl_pct','pct_chng'],axis=1)
    X_d = X_before
    X_d_forecast_out = X_d[-forecast_out:]
    X_d = X_d[:-forecast_out]
    
    y = np.array(teller_df['label'])
    y = y[:-forecast_out]
    y_d = np.array(teller_df['direction'])
    y_d = y_d[:-forecast_out]

    if idx % 50 == 0:
        print(idx,'counts name: ',name, '. Length of y: ', len(y))
    
    #model = SVR(kernel='linear',C=0.01).fit(X,y)
    model = LinearRegression().fit(X,y)
    #model = xgboost.XGBRegressor(max_depth=7).fit(X,y)
    #model_d = LogisticRegression(C=0.1).fit(X_d,y_d)
    model_d = xgboost.XGBClassifier(n_estimators=100,max_depth=7).fit(X_d,y_d)
    y_pred = model.predict(X_forecast_out)
    y_d_pred = model_d.predict(X_d_forecast_out)

    if y_d_pred:
        val = '+'
    else:
        val = '-'
   # val = '+'
    val += ' ' + str(float(y_pred))

    pred_lst.append(str(val))

0 counts name:  teller . Length of y:  149
50 counts name:  speeders . Length of y:  1660
100 counts name:  hairs . Length of y:  1660
150 counts name:  thyristors . Length of y:  1660
200 counts name:  jurisdictions . Length of y:  1660
250 counts name:  anchors . Length of y:  1660
300 counts name:  shovels . Length of y:  1660
350 counts name:  tab . Length of y:  1660
400 counts name:  tires . Length of y:  1660
450 counts name:  similarity . Length of y:  1660
500 counts name:  leaf . Length of y:  24


In [115]:
model_d.feature_importances_

array([ 0.19083969,  0.00763359,  0.        ,  0.        ,  0.        ,
        0.1221374 ,  0.        ,  0.        ,  0.03053435,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.08396947,
        0.01526718,  0.03816794,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.01526718,  0.        ,
        0.        ,  0.        ,  0.        ,  0.00763359,  0.02290076,
        0.01526718,  0.        ,  0.01526718,  0.        ,  0.08396947,
        0.        ,  0.        ,  0.03053435,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [116]:
# valid 채점용 데이터
pred_lst_2 = pred_lst.copy()
with open('price_rst.txt') as fp:
    price_rst = fp.read()
    price_rst = price_rst.splitlines()
    
for i in range(len(pred_lst)):
    pred_lst_2[i] = pred_lst[i].split(' ')
    
for i in range(len(price_rst)):
    price_rst[i] = price_rst[i].split(' ')

valid_df = pd.concat([pd.DataFrame(pred_lst_2), pd.DataFrame(price_rst)], axis=1)
valid_df.columns =['pred_0','pred_1','true_0','true_1']

valid_df['s2_1'] = 5-(valid_df['pred_1'].map(float) - valid_df['true_1'].map(float))**2
valid_df['s2'] = valid_df['s2_1'].map(lambda x: 0 if x <= 0 else x)

valid_df['s1'] = valid_df['pred_0'] == valid_df['true_0']
valid_df['s1'] = valid_df['s1'].map(lambda x: 5 if x == True else 0)

valid_df['score'] = valid_df['s1']+valid_df['s2']

10*valid_df['score'].sum()/len(valid_df)

79.97663992863288

In [ ]:
# logistic 74

# xgboost 79.08, X_d = X_before[['close','before_0','before_1']]
# X = preprocessing.scale(X_before[['close','hl_pct','pct_chng','before_0','before_1']])

# xgboost, before 14개
# 81.12, 실제 score 68

In [117]:
# submission
# 73.2714
output_file = 'submission.txt'

with open(output_file, 'w') as fp:
    for _test_pred in pred_lst:
        fp.write(_test_pred)
        fp.write('\n')

In [31]:
model_d.feature_importances_

array([ 0.13976377,  0.10137795,  0.09940945,  0.1003937 ,  0.07086615,
        0.11318897,  0.08759843,  0.08858268,  0.12204725,  0.07677165], dtype=float32)

In [ ]:
'close','hl_pct','pct_chng','before_0','before_1','direction_0','direction_1'